In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras import regularizers


from utils import windowed_dataset, build_basic_lstm,split_train_test,split_train_test_val
from config import *

import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt

import datetime

import mlflow
mlflow.set_tracking_uri("http://mlflow_server:5001")
tracking_uri = mlflow.get_tracking_uri()
print("Current tracking uri: {}".format(tracking_uri))

mpl.rcParams['figure.figsize'] = [35.0, 7.0]

today=datetime.date.today().strftime("%Y%m%d")
coefficients=17

rd_filename='tmp_residues_{}_{}.csv'.format(coefficients, today)
rd_feat_filename='tmp_residues_feat_{}_{}.csv'.format(coefficients, today)
rd_feat_norm_filename='tmp_residues_feat_norm_{}_{}.csv'.format(coefficients, today)

r_filename='tmp_radius_{}.csv'.format(today)
r_feat_filename='tmp_radius_feat_{}.csv'.format(today)
r_feat_norm_filename='tmp_radius_feat_norm_{}.csv'.format(today)

binary_labels_filename ='tmp_spiral_binaly_labels_{}.csv'.format(today)


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Current tracking uri: http://mlflow_server:5001


In [2]:
rd_df = pd.read_csv(rd_filename, index_col=0)
binary_labels = pd.read_csv(binary_labels_filename, index_col=0)

In [3]:
x=rd_df.values.astype('float32')
y=binary_labels.values.astype('int8').reshape(-1,1)
x.shape

(53, 4096)

Window function is applied to Dataset objects. Same windows lenghts for each time series corresponding to each subject are built asigning the rigth label. Then they are included in same Dataset object.

In [4]:
#windowing parameters
num_features = x.shape[1]
window_size = int(num_features/5 + 1)
shuffle_buffer = num_features
batch_size = 1

In [5]:
features=tf.data.Dataset.from_tensor_slices(x)
labels=(y)

In [6]:
#series, window_size, batch_size,label
for i,features in enumerate(x):
    new = windowed_dataset(features,window_size,batch_size,labels[i])
    if i>0:
        dataset = tf.data.Dataset.concatenate(dataset,new)
    else:
        dataset = new
        
dataset.cache()
dataset.element_spec

(TensorSpec(shape=(None,), dtype=tf.float32, name=None),
 TensorSpec(shape=(1,), dtype=tf.int8, name=None))

In [7]:
# DATASET_SIZE = int((num_features/window_size)*50)
# #TODO import from utils API
# train_size = int(0.7 * DATASET_SIZE)
# val_size = int(0.15 * DATASET_SIZE)
# test_size = int(0.15 * DATASET_SIZE)

# full_dataset = dataset
# full_dataset = full_dataset.shuffle(DATASET_SIZE, seed=seed)
# train_dataset = full_dataset.take(train_size).batch(mini_batch_size).prefetch(2).cache()
# test_dataset = full_dataset.skip(train_size)
# val_dataset = test_dataset.skip(test_size).batch(mini_batch_size).prefetch(2).cache()
# test_dataset = test_dataset.take(test_size).batch(mini_batch_size).prefetch(2).cache()

In [8]:
DATASET_SIZE = int((num_features/window_size)*50)
train_dataset, val_dataset, test_dataset = split_train_test_val(dataset, DATASET_SIZE, train_ratio=0.67, val_ratio=0.17, test_ratio=0.16, shuffle_buffer=DATASET_SIZE, seed=seed, mini_batch_size=mini_batch_size)

In [10]:
# #Early stop configuration
earlystop_callback = EarlyStopping(
  monitor='val_accuracy', min_delta=1e-3,
  patience=200)

training_earlystop_callback = EarlyStopping(
  monitor='accuracy', min_delta=1e-4,
  patience=200)

log_dir = "tensorflow_log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

def get_callbacks(name):
    return [
        earlystop_callback,
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, min_delta=1e-1),
      ]

def compile_and_fit(model, train_dataset, test_dataset, name, optimizer=None, max_epochs=1000):
    tf.keras.backend.clear_session()# avoid clutter from old models and layers, especially when memory is limited
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])
    model.summary()
    tf.random.set_seed(seed) # establecemos la semilla para tensorflow
    history = model.fit(train_dataset, 
                        use_multiprocessing=True, 
                        validation_data=test_dataset, epochs=max_epochs, 
                        callbacks=get_callbacks(name),
                        verbose=1, shuffle=True)
    return history

# Many models train better if you gradually reduce the learning rate during training. 
# Use optimizers.schedules to reduce the learning rate over time:
def get_optimizer(steps_per_epoch=1, lr=1e-4, multiplier=1000):
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(lr,
                                                                 decay_steps=steps_per_epoch*multiplier,
                                                                 decay_rate=1,
                                                                 staircase=False)
    return tf.keras.optimizers.Adam(lr_schedule)

In [12]:
size_histories = {}

In [13]:
with mlflow.start_run():
    mlflow.tensorflow.autolog()
    # LSTM basic architectures --> #TODO import from utils API
    tf.keras.backend.clear_session()# para evitar que entrenamientos annteriores afecten
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=1),input_shape=(window_size,)))
    model.add(tf.keras.layers.LSTM(8, activation=tf.nn.tanh, return_sequences=False))
    # model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

    size_histories['lstm/windows'] = compile_and_fit(model, train_dataset, 
                                                  val_dataset,
                                                  "fcnn/tiny", 
    #                                               optimizer=get_optimizer(),
                                                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
    #                                               optimizer = tf.keras.optimizers.SGD(lr=lr, momentum=0.9),
                                                  max_epochs=5)

2022/08/24 20:30:19 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 1, 820)            0         
                                                                 
 lstm (LSTM)                 (None, 8)                 26528     
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 26,537
Trainable params: 26,537
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
      1/Unknown - 2s 2s/step - loss: 0.1871 - accuracy: 0.7500WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0017s vs `on_train_batch_end` time: 0.0066s). Check your callbacks.
42/42 [==============================] - 69s 2s/step - loss: 0.0820 - 

INFO:tensorflow:Assets written to: /tmp/tmpyttlw7ap/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpyttlw7ap/model/data/model/assets


In [ ]:
print("\n#######################Evaluation###########################")
# Evaluate the model on the test data using `evaluate`
print('train acc:', max(size_histories['lstm/windows'].history["accuracy"]))
print('val acc:', max(size_histories['lstm/windows'].history["val_accuracy"]))

In [ ]:
plotter_loss = tfdocs.plots.HistoryPlotter(metric = 'loss', smoothing_std=1)
plotter_loss.plot(size_histories)
# plt.ylim(0., 1.)

In [ ]:
plotter_acc = tfdocs.plots.HistoryPlotter(metric = 'accuracy', smoothing_std=1)
plotter_acc.plot(size_histories)
# plt.ylim([0., 1.1])

In [ ]:
fig, axes = plt.subplots(4,1)
fig.tight_layout()

i=0
for (x,y),ax in zip(dataset.take(4), axes):
    ax.plot(x.numpy(), color=colors[y.numpy()[0]])
    ax.set_xticks(range(i,window_size,100))
    input_tensor=tf.expand_dims(x, axis=1)
    i+=1

In [ ]:
iter(dataset.filter(f).take(4).cache()).next()
prediction=model.predict(first[0])
prediction

In [ ]:
### model.evaluate(test_dataset)[1]